# **`cuSiril` Experimentation**

In [1]:
# IMPORT PACKAGES
from astropy.io import fits # FITS I/O
import ctypes # C/C++/CUDA shared libraries
import matplotlib.pyplot as plt # Matplotlib
import numpy as np # NumPy
from pysiril.siril import Siril # Siril app
from pysiril.wrapper import Wrapper # Siril API
import time # Timing

In [2]:
# LAUNCH SIRIL
app = Siril("/usr/bin/siril")
cmd = Wrapper(app)
app.Open()

INFO   : VERSION siril 1.2.0 : 
INFO   : Siril is compatible with pysiril
INFO   : Initialisation pySiril V0.0.15 : OK
INFO   : First step: Starting 'Siril -p' ....
INFO   : ThreadSiril is started
INFO   : run : /usr/bin/siril -p
INFO   : waiting: 
7s 
INFO   : Second step: Starting pipe reader ....
INFO   : ThreadReader is started
INFO   : Third step: Starting pipe writer ....
INFO   : PipeWriter is started
INFO   : Open()
> requires 1.2.0
: ready
: log: Running command: requires
> [status: success]


True

## Bias Frames

In [3]:
cmd.cd("/home/aanishpradhan/GitHub/cuSiril/assets/NGC-1499/Bias/")

> cd '/home/aanishpradhan/GitHub/cuSiril/assets/NGC-1499/Bias/'
: log: Running command: cd
: log: Setting CWD (Current Working Directory) to '/home/aanishpradhan/GitHub/cuSiril/assets/NGC-1499/Bias/'
> [status: success]


(True,)

Convert DSLR RAW frames to `.fits` files. Store the converted files into one
FITS file.

In [4]:
cmd.convertraw("Bias", debayer = True, fitseq = True)

> convertraw Bias -debayer -fitseq
: log: Running command: convertraw
: log: Convert: processing 5 files...
: log: Number of images allowed in the write queue: 4 (zero or less is unlimited)
: log: Decoding Nikon D5100 file (ISO=2000, Exposure=1/4000.0 sec)
: log: Filter pattern: RGGB
: log: Decoding Nikon D5100 file (ISO=2000, Exposure=1/4000.0 sec)
: log: Filter pattern: RGGB
: log: Decoding Nikon D5100 file (ISO=2000, Exposure=1/4000.0 sec)
: log: Filter pattern: RGGB
: log: Decoding Nikon D5100 file (ISO=2000, Exposure=1/4000.0 sec)
: log: Filter pattern: RGGB
: log: Filter Pattern: RGGB
: log: Filter Pattern: RGGB
: log: Filter Pattern: RGGB
: log: Filter Pattern: RGGB
: log: Reading RAW: file DSC_0146.NEF, 3 layer(s), 4948x3280 pixels
: log: Number of images allowed in the write queue: 17 (zero or less is unlimited)
: log: Reading RAW: file DSC_0145.NEF, 3 layer(s), 4948x3280 pixels
: log: writer: Saving image 0, 3 layer(s), 4948x3280 pixels, 16 bits
: log: Reading RAW: file DSC_0

(True,)

In [5]:
# View Bias subframes data
biasFrames = fits.open("/home/aanishpradhan/GitHub/cuSiril/assets/NGC-1499/Bias/Bias.fit")
print("Total frames:", len(biasFrames))
print("Image dimensions:", biasFrames[0].data.T.shape)
print("Data type:", biasFrames[0].data.dtype)
print("Maximum pixel value:", np.max(biasFrames[0].data))

Total frames: 5
Image dimensions: (4948, 3280, 3)
Data type: uint16
Maximum pixel value: 426


### Sum Stacking (w/o Input Normalization)

In [7]:
# Stack Bias subframes
cmd.stack("Bias", type = "sum", norm = "no", out = "Bias_stacked.fit")

> stack Bias sum -out=Bias_stacked.fit
: log: Running command: stack
: log: Checking sequences in the directory: /home/aanishpradhan/GitHub/cuSiril/assets/NGC-1499/Bias/.
: log: Stacking sequence Bias
: log: Not using rejection for stacking
: log: Processing all images in the sequence (5)
: log: Stacking result will be stored as a 32-bit image
: log: Sum stacking: with the current memory and thread limits, up to 5 thread(s) can be used
: log: Sum stacking: processing...
: log: Sequence processing succeeded.
: log: Execution time: 687.73 ms
: log: Integration of 5 images on 5 of the sequence:
: log: Pixel combination ......... normalized sum
: log: Input normalization ....... none
: log: Output normalization ...... disabled
: log: Pixel rejection ........... none
: log: Image weighting ........... disabled
: log: RGB equalization .......... disabled
: log: Background noise value (channel: #0): 936.074 (1.428e-02)
: log: Background noise value (channel: #1): 766.317 (1.169e-02)
: log: Ba

(True,)

In [9]:
# View master bias frame data
masterBiasFrame = fits.open("/home/aanishpradhan/GitHub/cuSiril/assets/NGC-1499/Bias/Bias_stacked.fit")
print("Total frames:", len(masterBiasFrame)) # Should be 1
print("Image size:", np.shape(masterBiasFrame[0].data.T)) # Should be same as above
print("Data type:", masterBiasFrame[0].data.dtype) # Big endian float32
print("Maximum pixel value:", np.max(masterBiasFrame[0].data))

Total frames: 1
Image size: (4948, 3280, 3)
Data type: >f4
Maximum pixel value: 1.0


In [10]:
x = np.empty(biasFrames[0].data.T.shape, dtype = biasFrames[0].data.dtype)
for i in range(0, len(biasFrames) ):
	x += np.copy(biasFrames[i].data.T)
x = x / np.max(x)

In [11]:
np.linalg.norm(masterBiasFrame[0].data.T - x)

6.086643852857138e-06

Sum stacking without input and output normalization involves adding the 
channels of the subframes independently and then dividing by the outright
largest pixel value.

## Light Frames

In [12]:
cmd.cd("/home/aanishpradhan/GitHub/cuSiril/assets/NGC-1499/Light/")

> cd '/home/aanishpradhan/GitHub/cuSiril/assets/NGC-1499/Light/'
: log: Running command: cd
: log: Setting CWD (Current Working Directory) to '/home/aanishpradhan/GitHub/cuSiril/assets/NGC-1499/Light/'
> [status: success]


(True,)

Convert DSLR RAW frames to `.fits` files. Store the converted files into one
FITS file.

In [13]:
cmd.convertraw("Light", debayer = True, fitseq = True)

> convertraw Light -debayer -fitseq
: log: Running command: convertraw
: log: Convert: processing 136 files...
: log: Number of images allowed in the write queue: 4 (zero or less is unlimited)
: log: Decoding Nikon D5100 file (ISO=2000, Exposure=49.3 sec)
: log: Filter pattern: RGGB
: log: Decoding Nikon D5100 file (ISO=2000, Exposure=49.3 sec)
: log: Filter pattern: RGGB
: log: Decoding Nikon D5100 file (ISO=2000, Exposure=49.3 sec)
: log: Filter pattern: RGGB
: log: Decoding Nikon D5100 file (ISO=2000, Exposure=49.1 sec)
: log: Filter pattern: RGGB
: log: Filter Pattern: RGGB
: log: Filter Pattern: RGGB
: log: Filter Pattern: RGGB
: log: Filter Pattern: RGGB
: log: Reading RAW: file DSC_0002.NEF, 3 layer(s), 4948x3280 pixels
: log: Number of images allowed in the write queue: 17 (zero or less is unlimited)
: log: Reading RAW: file DSC_0001.NEF, 3 layer(s), 4948x3280 pixels
: log: writer: Saving image 0, 3 layer(s), 4948x3280 pixels, 16 bits
: log: Reading RAW: file DSC_0003.NEF, 3 la

(True,)

In [14]:
# View Light subframes data
lightFrames = fits.open("/home/aanishpradhan/GitHub/cuSiril/assets/NGC-1499/Light/Light.fit")
print("Total frames:", len(lightFrames))
print("Image dimensions:", lightFrames[0].data.T.shape)
print("Data type:", lightFrames[0].data.dtype)
print("Maximum pixel value:", np.max(lightFrames[0].data))

Total frames: 136
Image dimensions: (4948, 3280, 3)
Data type: uint16
Maximum pixel value: 23577


### Calibration Frame Subtraction

Calibrate the light frames.

In [19]:
cmd.calibrate("Light", bias = "../Bias/Bias_stacked.fit", cfa = False, 
	equalize_cfa = False, fitseq = True)

: log: writer: Saving image 5, 3 layer(s), 4948x3280 pixels, 32 bits
: log: writer: Saving image 6, 3 layer(s), 4948x3280 pixels, 32 bits
: log: writer: Saving image 7, 3 layer(s), 4948x3280 pixels, 32 bits
: log: writer: Saving image 8, 3 layer(s), 4948x3280 pixels, 32 bits
: log: writer: Saving image 9, 3 layer(s), 4948x3280 pixels, 32 bits
: log: writer: Saving image 10, 3 layer(s), 4948x3280 pixels, 32 bits
: log: writer: Saving image 11, 3 layer(s), 4948x3280 pixels, 32 bits
: log: writer: Saving image 12, 3 layer(s), 4948x3280 pixels, 32 bits
: log: writer: Saving image 13, 3 layer(s), 4948x3280 pixels, 32 bits
: log: writer: Saving image 14, 3 layer(s), 4948x3280 pixels, 32 bits
: log: writer: Saving image 15, 3 layer(s), 4948x3280 pixels, 32 bits
: log: writer: Saving image 16, 3 layer(s), 4948x3280 pixels, 32 bits
: log: writer: Saving image 17, 3 layer(s), 4948x3280 pixels, 32 bits
: log: writer: Saving image 18, 3 layer(s), 4948x3280 pixels, 32 bits
: log: writer: Saving ima

(True,)

In [24]:
# View calibrated Light subframes data
calibratedLightFrames = fits.open("/home/aanishpradhan/GitHub/cuSiril/assets/NGC-1499/Light/pp_Light.fit", memmap = False)
print("Total frames:", len(calibratedLightFrames))
print("Image dimensions:", calibratedLightFrames[0].data.T.shape)
print("Data type:", calibratedLightFrames[0].data.dtype)

Total frames: 136
Image dimensions: (4948, 3280, 3)
Data type: >f4


In [82]:
np.linalg.norm(((lightFrames[0].data.T / 65535) - masterBiasFrame[0].data.T) - calibratedLightFrames[0].data.T)

7.086495648311434e-06

Calibration frame subtractions are done by dividing every light frame by 65535, 
the largest number that can be stored in a `uint16`, and then subtracting the 
master bias frame and then the master dark frame.

## **CUDA - `ctypes` Interfacing**

In [13]:
def flatten(subframes: np.ndarray) -> np.ndarray:
    pixelsPerImage = subframes.shape[0] * subframes.shape[1]
    numberOfSubframes = subframes.shape[2]
    
    flattenedSubframes = np.empty((1, pixelsPerImage * numberOfSubframes), dtype=subframes.dtype)
    
    for subframe in range(0, numberOfSubframes):
        start_idx = subframe * pixelsPerImage
        end_idx = (subframe + 1) * pixelsPerImage
        flattenedSubframes[0, start_idx:end_idx] = subframes[:, :, subframe].flatten()

    return flattenedSubframes.flatten()

flattenedRedChannel = flatten(biasFrameRedChannel)
flattenedGreenChannel = flatten(biasFrameGreenChannel)
flattenedBlueChannel = flatten(biasFrameBlueChannel)

In [36]:
lib = ctypes.CDLL("./Stacking_Kernels.so")
sumStack = lib.sumStack
tic2 = time.time()
sumStack(ctypes.c_uint64(5), ctypes.c_uint64(imageWidth), ctypes.c_uint64(imageHeight), 
	ctypes.c_void_p(flattenedRedChannel.ctypes.data), 
	ctypes.c_void_p(flattenedGreenChannel.ctypes.data), 
	ctypes.c_void_p(flattenedBlueChannel.ctypes.data), 
	ctypes.c_void_p(stackedRedChannel.ctypes.data), 
	ctypes.c_void_p(stackedGreenChannel.ctypes.data), 
	ctypes.c_void_p(stackedBlueChannel.ctypes.data))
toc2 = time.time()

Red max is 1374
Atomic max for red is 1374
Atomic max for green is 1206
Atomic max for green is 1206
